**Run the following two cells before you begin.**

In [ ]:
%autosave 10

In [1]:
import pandas as pd
import numpy as np

______________________________________________________________________
**First, import your data set and define the sigmoid function.**
<details>
    <summary>Hint:</summary>
    The definition of the sigmoid is $f(x) = \frac{1}{1 + e^{-X}}$.
</details>

In [2]:
# Import the data set
url='https://raw.githubusercontent.com/Technocolabs100/Project-Data-Set-Repository/master/Data%20set/cleaned_data.csv'
df = pd.read_csv(url, error_bad_lines=False)
df.head()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_1,PAY_2,PAY_3,PAY_4,...,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month,EDUCATION_CAT,graduate school,high school,others,university
0,798fc410-45c1,20000,2,2,1,24,2,2,-1,-1,...,0,0,0,0,1,university,0,0,0,1
1,8a8c8f3b-8eb4,120000,2,2,2,26,-1,2,0,0,...,1000,1000,0,2000,1,university,0,0,0,1
2,85698822-43f5,90000,2,2,2,34,0,0,0,0,...,1000,1000,1000,5000,0,university,0,0,0,1
3,0737c11b-be42,50000,2,2,1,37,0,0,0,0,...,1200,1100,1069,1000,0,university,0,0,0,1
4,3b7f77cc-dbc0,50000,1,2,1,57,-1,0,-1,0,...,10000,9000,689,679,0,university,0,0,0,1


In [56]:
# Define the sigmoid function
def sigmoid(X):
    Y = 1 / (1 + np.exp(-X))
    return Y

**Now, create a train/test split (80/20) with `PAY_1` and `LIMIT_BAL` as features and `default payment next month` as values. Use a random state of 24.**

In [27]:
# Create a train/test split
from sklearn.model_selection import train_test_split
X = df.iloc[:,[1,6]]
y = df['default payment next month'].values
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=24)

______________________________________________________________________
**Next, import LogisticRegression, with the default options, but set the solver to `'liblinear'`.**

In [44]:
from sklearn.linear_model import LogisticRegression
model=LogisticRegression(solver = 'liblinear',random_state=24)

______________________________________________________________________
**Now, train on the training data and obtain predicted classes, as well as class probabilities, using the testing data.**

In [45]:
# Fit the logistic regression model on training data
model.fit(X,y)

LogisticRegression(random_state=24, solver='liblinear')

In [36]:
# Make predictions using `.predict()`
y_pred=model.predict(X_test)
y_pred

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [37]:
# Find class probabilities using `.predict_proba()`
y_pred_prob=model.predict_proba(X_test)
y_pred_prob

array([[0.74974322, 0.25025678],
       [0.58489201, 0.41510799],
       [0.79763112, 0.20236888],
       ...,
       [0.58489201, 0.41510799],
       [0.82881998, 0.17118002],
       [0.66502685, 0.33497315]])

______________________________________________________________________
**Then, pull out the coefficients and intercept from the trained model and manually calculate predicted probabilities. You'll need to add a column of 1s to your features, to multiply by the intercept.**

In [78]:
# Add column of 1s to features
ones_and_features = np.hstack([np.ones((X_test.shape[0],1)), X_test])
ones_and_features


array([[ 1.0e+00,  1.6e+05,  2.0e+00],
       [ 1.0e+00,  5.0e+04,  1.0e+00],
       [ 1.0e+00,  2.0e+05, -1.0e+00],
       ...,
       [ 1.0e+00,  5.0e+04, -1.0e+00],
       [ 1.0e+00,  2.3e+05,  1.0e+00],
       [ 1.0e+00,  1.0e+05,  2.0e+00]])

In [80]:
# Get coefficients and intercepts from trained model
intercept_and_coefs = np.concatenate([model.intercept_.reshape(1,1), model.coef_], axis=1)
intercept_and_coefs

array([[-6.59908540e-11, -6.85777045e-06,  8.18609439e-11]])

In [81]:
# Manually calculate predicted probabilities
X_lin_comb = np.dot(intercept_and_coefs, np.transpose(ones_and_features))
y_pred_proba_manual = sigmoid(X_lin_comb)

______________________________________________________________________
**Next, using a threshold of `0.5`, manually calculate predicted classes. Compare this to the class predictions output by scikit-learn.**

In [82]:
# Manually calculate predicted classes
y_pred_manual = y_pred_proba_manual >= 0.5

In [83]:
# Compare to scikit-learn's predicted classes
np.array_equal(y_pred.reshape(1,-1),y_pred_manual)

True

______________________________________________________________________
**Finally, calculate ROC AUC using both scikit-learn's predicted probabilities, and your manually predicted probabilities, and compare.**

In [85]:
# Use scikit-learn's predicted probabilities to calculate ROC AUC
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,y_pred_proba_manual.reshape(y_pred_proba_manual.shape[1],))

0.627207450280691

In [87]:
# Use manually calculated predicted probabilities to calculate ROC AUC
roc_auc_score(y_test,y_pred_prob[:,1])

0.627207450280691